In [ ]:

from __future__ import print_function



from collections import namedtuple
import ctypes
import math
import struct

import roslib.message
from sensor_msgs.msg import PointCloud2, PointField

_DATATYPES = {}
_DATATYPES[PointField.INT8]    = ('b', 1)
_DATATYPES[PointField.UINT8]   = ('B', 1)
_DATATYPES[PointField.INT16]   = ('h', 2)
_DATATYPES[PointField.UINT16]  = ('H', 2)
_DATATYPES[PointField.INT32]   = ('i', 4)
_DATATYPES[PointField.UINT32]  = ('I', 4)
_DATATYPES[PointField.FLOAT32] = ('f', 4)
_DATATYPES[PointField.FLOAT64] = ('d', 8)

def read_points(cloud, field_names=None, skip_nans=False, uvs=[]):

    assert isinstance(cloud, roslib.message.Message) and cloud._type == 'sensor_msgs/PointCloud2', 'cloud is not a sensor_msgs.msg.PointCloud2'
    fmt = _get_struct_fmt(cloud.is_bigendian, cloud.fields, field_names)
    width, height, point_step, row_step, data, isnan = cloud.width, cloud.height, cloud.point_step, cloud.row_step, cloud.data, math.isnan
    unpack_from = struct.Struct(fmt).unpack_from

    if skip_nans:
        
        for v in range(height):
            offset = row_step * v
            for u in range(width):
                p = unpack_from(data, offset)
                has_nan = False
                for pv in p:
                    if isnan(pv):
                        has_nan = True
                        break
                if not has_nan:
                    yield p
                offset += point_step
    

def _get_struct_fmt(is_bigendian, fields, field_names=None):
    fmt = '>' if is_bigendian else '<'

    offset = 0
    for field in (f for f in sorted(fields, key=lambda f: f.offset) if field_names is None or f.name in field_names):
        if offset < field.offset:
            fmt += 'x' * (field.offset - offset)
            offset = field.offset
        if field.datatype not in _DATATYPES:
            print('Skipping unknown PointField datatype [%d]' % field.datatype, file=sys.stderr)
        else:
            datatype_fmt, datatype_length = _DATATYPES[field.datatype]
            fmt    += field.count * datatype_fmt
            offset += field.count * datatype_length

    return fmt

In [4]:
import glob
import os
import sys
import time
import random
import math


import carla

actor_list = []


def generate_lidar_blueprint(blueprint_library):
    lidar_blueprint = blueprint_library.find('sensor.lidar.ray_cast_semantic')
    lidar_blueprint.set_attribute('channels', str(32))
    #write the code for set points_per_second
    lidar_blueprint.set_attribute("points_per_second",str(56000))
    #write the code for set rotation frequency
    lidar_blueprint.set_attribute("rotation_frequency",str(10))
    #write the code for set range
    lidar_blueprint.set_attribute("range",str(10))
    lidar_blueprint.set_attribute("upper_fov",str(10))
    lidar_blueprint.set_attribute("lower_fov",str(-10))
    return lidar_blueprint


object_id = {"None": 0,
             "Buildings": 1,
             "Fences": 2,
             "Other": 3,
             "Pedestrians": 4,
             "Poles": 5,
             "RoadLines": 6,
             "Roads": 7,
             "Sidewalks": 8,
             "Vegetation": 9,
             "Vehicles": 10,
             "Wall": 11,
             "TrafficsSigns": 12,
             "Sky": 13,
             "Ground": 14,
             "Bridge": 15,
             "RailTrack": 16,
             "GuardRail": 17,
             "TrafficLight": 18,
             "Static": 19,
             "Dynamic": 20,
             "Water": 21,
             "Terrain": 22
             }

key_list = list(object_id.keys())
value_list = list(object_id.values())


def semantic_lidar_data(point_cloud_data,car):
    distance_name_data = {}
    for detection in point_cloud_data:
        # print(detection)
        position = value_list.index(detection.object_tag)
        distance = math.sqrt((detection.point.x ** 2) + (detection.point.y ** 2) + (detection.point.z ** 2))
        distance_name_data["distance"] = distance
        distance_name_data["name"] = key_list[position]
        #write code here to display only name of object
        #print(f"Name of all objects nearby car  : - {distance_name_data['name']}  {detection.point.x}  {detection.point.y}  {detection.point.z}")
        world.debug.draw_string(carla.Location(x=detection.point.x+car.get_location().x,y=detection.point.y+car.get_location().y,z=detection.point.z+car.get_location().z), '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=30.0,
            persistent_lines=True)


client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
map = world.get_map()
get_blueprint_of_world = world.get_blueprint_library()
car_model = get_blueprint_of_world.filter('model3')[0]
spawn_point = (world.get_map().get_spawn_points()[0])
dropped_vehicle = world.spawn_actor(car_model, spawn_point)
lidar_sensor = generate_lidar_blueprint(get_blueprint_of_world)
sensor_lidar_spawn_point = carla.Transform(carla.Location(x=0, y=0, z=2.0),
                                           carla.Rotation(pitch=0.000000, yaw=0.0, roll=0.000000))
sensor = world.spawn_actor(lidar_sensor, sensor_lidar_spawn_point, attach_to=dropped_vehicle)
#dropped_vehicle.apply_control(carla.VehicleControl(throttle=0.5, steer=0))
sensor.listen(lambda point_cloud_data: semantic_lidar_data(point_cloud_data,dropped_vehicle))

In [5]:
import carla
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [ ]:
25.153568267822266  -26.13261604309082  6.395630836486816    Vegetation

In [3]:
import carla
client = carla.Client('127.0.0.1', 2000)
client.set_timeout(10.0)
world = client.get_world()
spawn_point = world.get_map().get_spawn_points()[0].location
world.debug.draw_string(carla.Location(x=25.153568267822266+spawn_point.x,y=-26.13261604309082+spawn_point.y,z=6.395630836486816+spawn_point.z), '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=30.0,
        persistent_lines=True)
